# "The sed command (macOS)"
> "The sed command is a stream text editor, capable of performing searches and substitution, and supports regular expressions"

- toc: true 
- badges: true
- comments: true
- categories: [cheatsheet, cli, macOS]

# Preliminaries

## References
- [FreeBSD man page](https://www.freebsd.org/cgi/man.cgi?query=sed) (freebsd)
- [mac OS sed documentation](https://ss64.com/osx/sed.html) (ss64)
- [sed tutorial](https://www.grymoire.com/Unix/Sed.html) (grymoire)
- [IBM: the `sed` command](https://www.ibm.com/support/knowledgecenter/en/ssw_aix_72/s_commands/sed.html)

## BSD `sed`
>Important: MacOS uses the FreeBSD version of `sed`:

In [1]:
man sed | grep -n "BSD" -C1

1-
2:SED(1)                    BSD General Commands Manual                   SED(1)
3-
--
--
345-
346:     The -E, -a and -i options are non-standard FreeBSD extensions and may not
347-     be available on other operating systems.
--
--
352-AUTHORS
353:     Diomidis D. Spinellis <dds@FreeBSD.org>
354-
--
--
360-
361:BSD                              May 10, 2005                              BSD


>Note: There doesn't seem to be a way to find a version number on macOS from the OS, even if one looks in the binary code with the command `strings $(which sed)`.  However, this information is available from [apple's source browser](https://opensource.apple.com/source/text_cmds/).

# Basic usage

## Silent mode with flag`-n`, print with command `p`

By default, the command `sed` simply outputs each line, one at a time:

In [185]:
echo -e "bob\njoe\ntim" | sed

bob
joe
tim


Do not print anything:

In [186]:
echo -e "bob\njoe\ntim" | sed -n

Duplicate every line, i.e. for each line, print it:

In [187]:
echo -e "bob\njoe\ntim" | sed 'p'

bob
bob
joe
joe
tim
tim


Print each line once:

In [188]:
echo -e 'bob\njoe\ntim' | sed -n 'p'

bob
joe
tim


Reverse command with '!':

In [190]:
echo -e 'bob\njoe\ntime' | sed -n '!p'

## Pattern matching
Print lines matching a pattern with the command `p`:

In [78]:
echo -e "bob rob bob\njoe\nbob" | sed -n '/bob/p'

bob rob bob
bob


Some explanations:
- `-n`: do not print anything;
- `/bob/`: pattern to match;
- `/p`: print line when there is a match.

Print lines _not_ matching a pattern:

In [192]:
echo -e "bob rob bob\njoe\nbob\ntim" | sed -n '/bob/ !p'

joe
tim


Print line numbers where matches occur with the command `=`:

In [86]:
echo -e "bob rob bob\njoe\nbob" | sed -n '/bob/='

1
3


## Quit with command `q`

Quit printing file after match:

In [196]:
echo -e 'one: bob\ntwo: bob\nthree:bob\n' | sed -e '/two/ q'

one: bob
two: bob


As an application, perform substitution up to match: 

In [200]:
echo -e 'one: bob\ntwo: bob\nthree:bob\n' | sed -e 's/bob/BOB/' -e '/two/ q'

one: BOB
two: BOB


Note how the order matters;

In [201]:
echo -e 'one: bob\ntwo: bob\nthree:bob\n' | sed -e '/two/ q' -e 's/bob/BOB/'

one: BOB
two: bob


## Transformation with `y`:

In [87]:
echo "This is NOW in loweRcase" | sed 'y/ABCDEFGHIJKLMNOPQRSTUVWXYZ/abcdefghijklmnopqrstuvwxyz/'

this is now in lowercase


## Substitution with `s`

Sustitute a word for another, in each line:

In [2]:
echo -e "day\nDay\ndAy\nMonday" | sed 's/day/night/'

night
Day
dAy
Monnight


Note that the match is case-sensitive.

Only the first occurrence in each line will be substituted:

In [3]:
echo -e "one two three, one two three \nfour three two one\none hundred" | sed 's/one/ONE/'

ONE two three, one two three 
four three two ONE
ONE hundred


## Select occurrence for substitution with index: `/1`, `/2`, etc.

In [4]:
echo "one one one" | sed 's/one/ONE/1'

ONE one one


In [5]:
echo "one one one" | sed 's/one/ONE/2'

one ONE one


In [6]:
echo "one one one" | sed 's/one/ONE/3'

one one ONE


In [7]:
echo "one one one" | sed 's/one/ONE/4'

one one one


## Global substitution: `/g`

To perform substitution on all non-overlapping match, use the `/g` option:

In [8]:
echo -e "one two three, one two three \nfour three two one\none hundred" | sed 's/one/ONE/g'

ONE two three, ONE two three 
four three two ONE
ONE hundred


## Silence output with `-n`, printed substituted lines with `/p`
The flag `-n`, nothing is sent to standard output:

In [9]:
echo -e "day\nDay\ndAy\nMonday" | sed -n 's/day/night/'

But with the `/p` flag, write the pattern space to standard output if a substitution occurred:

In [10]:
echo -e "day\nDay\ndAy\nMonday" | sed -n 's/day/night/p'

night
Monnight


Thus, without the option `-n`, lines with substitution appear twice:

In [11]:
echo -e "day\nDay\ndAy\nMonday" | sed 's/day/night/p'

night
night
Day
dAy
Monnight
Monnight


## Regular expressions

Replace the first single-digit number with a fixed string:

In [12]:
echo "abce345f" | sed 's/[0-9]/X/'

abceX45f


Replace the first longest consecutive string of digits with a fixed string:

In [13]:
echo "123 456 abc" | sed 's/[0-9]*/X/'

X 456 abc


>Important: Since the asterisk `*` stands for **zero** or more character, the first match for `[0-9]*` will occur at the beginning of the line if it doesn't start with a number:

In [14]:
echo "abc 123 def " | sed 's/[0-9]*/X/'

Xabc 123 def 


To match **one** or more character, use the extended regular expressioon `+`, with the option `-E`:

In [15]:
echo "abc 123 def " | sed -E 's/[0-9]+/X/'

abc X def 


## Refer to the match with `&`

Put parentheses around the first digit appearing in the line:

In [19]:
echo "abc 123 def " | sed -E 's/[0-9]/(&)/'

abc (1)23 def 


Put parentheses around the first multi-digit number appearing in the line:

In [21]:
echo "abc 123 def " | sed -E 's/[0-9]+/(&)/'

abc (123) def 


Put parentheses around each digit appearing in the line:

In [23]:
echo "abc 123 def 456" | sed -E 's/[0-9]/(&)/g'

abc (1)(2)(3) def (4)(5)(6)


`&` can be used multiple times:

In [24]:
echo "abc 123 def" | sed -E 's/[0-9]+/(& &)/'

abc (123 123) def


## Handle on matches: `(` `)`, `\1`, `\2`, etc.

Keep only first word of line, using a basic regular expression:

In [37]:
echo "Do not do this" | sed 's/\([a-zA-Z]*\).*/\1/'

Do


... and using an extenced regular expression:

In [38]:
echo "Do not do this" | sed -E 's/([a-zA-Z]*).*/\1/'

Do


Swap strings:

In [39]:
echo "123 321: swap occurred" | sed -E 's/([0-9]+) ([0-9]+)/\2 \1/'

321 123: swap occurred


Remove duplicated words:

In [46]:
echo "I I don't stutter stutter" | sed 's/\([a-zA-Z]*\) \1/\1 /g'

I  don't stutter 


In [49]:
echo "I I don't stutter stutter" | sed 's/\([a-zA-Z]*\) \1/\1 /g'

I  don't stutter 


Parentheses around first word:

In [51]:
echo "put parentheses around first blob" | sed 's/[^ ]*/(&)/'

(put) parentheses around first blob


Apply substitution on second occurrence only:

In [55]:
echo "au invisibleword clair de la lune" | sed -E 's/[a-z]+//2'

au  clair de la lune


Add a colon after character in position `10`:

In [59]:
echo "add a colon after character in position ten" | sed 's/./&:/10'

add a colo:n after character in position ten


# Duplicate `grep` with `-n` and `/p`
>Note: `grep` looks for the first occurrence of a pattern in each line:

In [65]:
echo -e "bob rob bob\njoe\nbob" | grep "bob"

bob rob bob
bob


In [64]:
echo -e "bob rob bob\njoe\nbob" | sed -n 's/bob/&/p'

bob rob bob
bob


In [67]:
echo -e "bob rob bob\njoe\nbob" | sed -n '/bob/p'

bob rob bob
bob


# Execute several commands: `-e`

In [96]:
echo "abc" | sed -n -e 's/a/1/p' -e 's/b/2/p'

1bc
12c


## Delete lines

Delete line by its number:

In [99]:
echo -e "1\n2\n3" | sed '2d'

1
3


Delete last line:

In [100]:
echo -e "1\n2\n3" | sed '$d'

1
2


In [105]:
echo -e 'one\n\ntwo\n\nthree' | sed '/\S/d'

one

two

three


## Combining  substitution flags

In [115]:
echo -e "one\none one\none one one\none one one one" | sed -n 's/one/ONE/3p'

one one ONE
one one ONE one


## Restricting to a line number

In [118]:
echo -e 'one\none\none' | sed '2 s/one/ONE/'

one
ONE
one


## Match and substitution 

In [123]:
echo -e "bob: hi\njoe: hello\nbob: how are you?" | sed '/bob/ s/h/H/'

bob: Hi
joe: hello
bob: How are you?


In [128]:
echo -e 'parentheses around first word of each lines\nonly if the word "line"\nappears in it\nthis is the last line' | sed -E  '/line/s/[a-z]+/(&)/'

(parentheses) around first word of each lines
(only) if the word "line"
appears in it
(this) is the last line


# Further examples

In [129]:
echo -e '#a\nthis line does not start with a pound sign\n#b' | sed 's/^#.*/[There was a line beginning with a pound sign]/'

[There was a line beginning with a pound sign]
this line does not start with a pound sign
[There was a line beginning with a pound sign]


Count the number of non-blank lines not beginning with `#`:

In [131]:
echo -e "#a\nthis line does not start with a pound sign\n#b" | sed 's/^#.*//' | grep "\S" | wc -l

       1


## Ranges by line number

Specify range of lines to which substitution is performed:

In [133]:
echo -e "one\none\none\none" | sed '2,3 s/one/ONE/'

one
ONE
ONE
one


Perform subsitution from specified line to end of file:

In [134]:
echo -e "one\none\none\none" | sed '2,$ s/one/ONE/'

one
ONE
ONE
ONE


## Ranges by patterns

In [146]:
echo -e "one: bob\ntwo: bob\nthree: bob\nfour: bob\nfive: bob" | sed '/two/,/three/ s/bob/BOB/'

one: bob
two: BOB
three: BOB
four: bob
five: bob


## Hybrid ranges

In [147]:
echo -e "one: bob\ntwo: bob\nthree: bob\nfour: bob\nfive: bob" | sed '2,/three/ s/bob/BOB/'

one: bob
two: BOB
three: BOB
four: bob
five: bob


Remove everything up to the first blank line:

In [180]:
echo -e "first line\nsecond line\n\nthere was a blank line\nright above\nthe end" | sed '1,/^$/d'

there was a blank line
right above
the end


In [171]:
echo -e 'a\n\nb c' | grep -n --color '\s'

3:b c


In [157]:
echo -e "first line\nsecond line\n\nthere was a blank line\nthe end" | hexdump -C

00000000  66 69 72 73 74 20 6c 69  6e 65 0a 73 65 63 6f 6e  |first line.secon|
00000010  64 20 6c 69 6e 65 0a 0a  74 68 65 72 65 20 77 61  |d line..there wa|
00000020  73 20 61 20 62 6c 61 6e  6b 20 6c 69 6e 65 0a 74  |s a blank line.t|
00000030  68 65 20 65 6e 64 0a                              |he end.|
00000037


In [170]:
echo -e '\t' | hexdump -C

00000000  09 0a                                             |..|
00000002
